[exercises](ir.ipynb)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.linalg import convolution_matrix
import soundfile as sf
import tools

In [ ]:
sig, fs = sf.read('data/xmas.wav')
fs

In [ ]:
sig_blackbox = tools.blackbox(sig, fs)

In [ ]:
sf.write('data/xmas_blackbox.wav', sig_blackbox, fs)

<audio src="data/xmas.wav" controls>Your browser does not support the audio element.</audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_blackbox.wav" controls>Your browser does not support the audio element.</audio>
[data/xmas_blackbox.wav](data/xmas_blackbox.wav)

The system is supposed to sound like narrow-band telephony (limited to a frequency range from 300 to 3400 Hz).

In [ ]:
dur = 1/10  # seconds
imp = np.zeros(np.ceil(dur * fs).astype('int'))
imp[0] = 1

In [ ]:
ir = tools.blackbox(imp, fs)

In [ ]:
t = np.arange(len(imp)) / fs
plt.plot(t, ir)
plt.xlabel('t / s')
plt.ylabel('ir(t)')
plt.figure()
plt.plot(t, tools.db(ir))
plt.xlabel('t / s')
plt.ylabel('20 log10 |ir(t)| in dB')
plt.ylim(-130, -10)

In [ ]:
w, h = signal.freqz(ir)
plt.figure()
plt.plot(w * fs / (2 * np.pi), tools.db(h))  # logarithmic y axis
# plt.plot(w * fs / (2 * np.pi), np.abs(h))  # linear y axis
plt.xscale('log')
plt.xlabel('f / Hz')
plt.ylabel('20 log10 |ir(t)| in dB')
plt.ylim(-100, 10)
plt.grid(True)

In [ ]:
def naive_convolution(x, h):
    """Very inefficient convolution of two one-dimensional arrays."""
    totalsize = len(x) + len(h) - 1
    y = np.zeros(totalsize)  # Allocate and initialize memory
    for n in range(totalsize):
        # m must be within range(len(x)) and it must meet the condition
        # n >= m and (n-m) < len(h)
        m_start = max(n + 1 - len(h), 0)
        m_stop = min(n + 1, len(x))
        for m in range(m_start, m_stop):
            y[n] += x[m] * h[n - m]
    return y

In [ ]:
# This takes a loooong time:
#sig_naive = naive_convolution(sig, ir)

# Let's just try a tiny bit of the signal:
%time sig_naive = naive_convolution(sig[:1000], ir)

`len(y) == len(x) + len(h) - 1`

The result should obviously be the same (but with `blackbox()` the resulting signal is shorter because the end is cut off).

In [ ]:
ir.shape

In [ ]:
# linear convolution via toeplitz matrix
A = convolution_matrix(ir, len(sig[:1000]), mode='full')
print(A.shape)
%time sig_toeplitz = A @ sig[:1000]
print(np.allclose(sig_naive, sig_toeplitz))

In [ ]:
# fast convolution with fft
x = np.zeros(len(sig_naive))  # we need zeropadding for linear conv
h = np.zeros(len(sig_naive))
x[:1000] = sig[:1000]
h[:len(ir)] = ir
%time sig_fft = np.fft.ifft(np.fft.fft(x) * np.fft.fft(h))
print(np.allclose(sig_naive, sig_fft))

Yes, it should indeed be faster. *Very very much* faster! Note that we take the whole `sig` signal below.

In [ ]:
%time sig_convolve = np.convolve(sig, ir)

In [ ]:
%time sig_fftconvolve = signal.fftconvolve(sig, ir)

Note that with short durations, the result of `%time` isn't really reliable, because the measurement may be influenced by other things going on before and after the actual execution of the function and other processes executed by the operating system at the same time.

To get a more reliable estimation, use `%timeit` instead.

In [ ]:
# TODO: longer signal for more realistic comparison

In [ ]:
sig_blackbox_nonlinear = tools.blackbox_nonlinear(sig, fs)

In [ ]:
sf.write('data/xmas_blackbox_nonlinear.wav', sig_blackbox_nonlinear, fs)

The non-linear system should sound like a distorted narrow-band telephone.

In [ ]:
ir_nonlinear = tools.blackbox_nonlinear(imp, fs)

In [ ]:
sig_nonlinear_convolve = np.convolve(sig, ir_nonlinear)

In [ ]:
sf.write('data/xmas_convolution_nonlinear.wav', sig_nonlinear_convolve, fs)

<audio src="data/xmas.wav" controls>Your browser does not support the audio element.</audio>
[original](data/xmas.wav)

<audio src="data/xmas_blackbox.wav" controls>Your browser does not support the audio element.</audio>
[through `tools.blackbox()`](data/xmas_blackbox.wav)

<audio src="data/xmas_blackbox_nonlinear.wav" controls>Your browser does not support the audio element.</audio>
[through `tools.blackbox_nonlinear()`](data/xmas_blackbox_nonlinear.wav)

<audio src="data/xmas_convolution_nonlinear.wav" controls>Your browser does not support the audio element.</audio>
[trying to use the impulse response of the non-linear system (doesn't work!)](data/xmas_convolution_nonlinear.wav)

The answer to the final question is: No!

In [ ]:
sig, fs = sf.read('data/singing.wav')
sig_compressed = tools.compressor(sig, threshold=-30, ratio=3, makeup_gain=12)
sf.write('data/singing_compressed.wav', sig_compressed, fs)

In [ ]:
time = np.arange(len(sig)) * (1/fs)

plt.plot(time, sig_compressed, label='compressed')
plt.plot(time, sig, label='original')
plt.xlabel('t / s')
plt.legend()

<audio src="data/singing.wav" controls>Your browser does not support the audio element.</audio>
[original](data/singing.wav)

<audio src="data/singing_compressed.wav" controls>Your browser does not support the audio element.</audio>
[through `tools.compressor()`](data/singing_compressed.wav)

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>